# Projekt 1 - Nimby - Paweł Malisz, Mikołaj Białek - Ocena: 5

## 1. Cel ćwiczenia

Celem ćwiczenia jest modyfikacja zaproponowanej w bibliotece easyAI implementacji gry Nim, tak aby zawierała element losowy. Losowość polegać będzie na tym, że przy wykonywaniu ruchu jest 10. procentowe prawdopodobieństwo, że gracz bierze ze stosu o 1 elemetn mniej niż zamierzał. Zmodyfikowana gra zostanie wielokrotnie uruchomiona (w wersjach deterministycznej i probabilistycznej) z dwoma graczami AI, dla dwóch różnych głębokości, a otrzymane wyniki (ilość wygranych, przegranych, remisów) porównane. Następnie gra zostanie wielokrotnie uruchomiona dla dwóch różnych algorytmów (Negamax z i bez odcięcia alfa-beta) z dwoma ustawieniami maksymalnej głębokości. Wyniki zostaną porównane, a dodatkowo policzony zostanie średni czas spędzony na wybraniu akcji przez każdy wariant AI. Na końcu gra zostanie uruchomiona z algorytmem expecti-minimax z odcięciem alfa-beta, a jego wydajność zostanie porównana z wcześniej użytymi algorytmami.

## 2. Opis problemu

Nim to gra dla dwóch osób z użyciem pinonków, które dzieli się na kupki dowolnej wielkości. Następnie gracze zabierają na zmianę dowolną, niezerową liczbę pionków. W jednym ruchu można zebrać tylko z jednej kupki. Wygrywa gracz, który zabiera ostatni pionek.

Przez naturę samej gry, jeśli obaj gracze grają optymalnie, można z góry stwierdzić, który z nich wygra. W takim razie więc, jeśli dla zadanej planszy, 2 algorytmy AI będą grały optymalnie zawsze wygra ten sam. Czy tak się realnie stanie zobaczymy później, po wykonaniu testów. Dodanie element losowego powinno zatem zmienić wyniki, w stosunku do wariantu deterministycznego gry. Testy będziemy przeprowadzać dla stałej planszy, składającej się z 4 kupek pionków o odpowiednich ilościach: 2, 3, 4, 5.

## 3. Realizacja rozwiązania

### Kod gry Nim z https://github.com/Zulko/easyAI
Usunęliśmy funkcję `unmake_move` (funkcja ta pszyspiesza nieco działanie algorytmu, ale kiedy jest ona obecna niżej zaprezentowana przez nas wersja probabilistyczna gry nie działa) oraz dodaliśmy do konstruktora parametr `current_player`, tak aby dało się zmieniać gracza, rozpoczynającego rozgrywkę.

In [1]:
from easyAI import TwoPlayerGame


class Nim(TwoPlayerGame):
    """
    The game starts with 4 piles of 5 pieces. In turn the players
    remove as much pieces as they want, but from one pile only. The
    player that removes the last piece loses.
    Parameters
    ----------
    players
      List of the two players e.g. [HumanPlayer(), HumanPlayer()]
    piles:
      The piles the game starts with. With piles=[2,3,4,4] the
      game will start with 1 pile of 2 pieces, 1 pile of 3 pieces, and 2
      piles of 4 pieces.
    max_removals_per_turn
      Max number of pieces you can remove in a turn. Default is no limit.
    """

    def __init__(self, players=None, max_removals_per_turn=None, piles=(2, 3, 4, 5), current_player=1):
        """ Default for `piles` is 5 piles of 5 pieces. """
        self.players = players
        self.piles = list(piles)
        self.max_removals_per_turn = max_removals_per_turn
        self.current_player = current_player

    def possible_moves(self):
        return [
            "%d,%d" % (i + 1, j)
            for i in range(len(self.piles))
            for j in range(
                1,
                self.piles[i] + 1
                if self.max_removals_per_turn is None
                else min(self.piles[i] + 1, self.max_removals_per_turn),
            )
        ]

    def make_move(self, move):
        move = list(map(int, move.split(",")))
        self.piles[move[0] - 1] -= move[1]

    def show(self):
        print(" ".join(map(str, self.piles)))

    def win(self):
        return max(self.piles) == 0

    def is_over(self):
        return self.win()

    def scoring(self):
        return 100 if self.win() else 0

    def ttentry(self):
        return tuple(self.piles)  # optional, speeds up AI

### Wersja gry z 10% szansą wzięcia 1 elementu mniej niż zamierzano

In [2]:
import random

class MissNim(Nim):
    def make_move(self, move):
        move = list(map(int, move.split(",")))
        m = move[1] - 1 if random.randint(1, 10) == 1 else move[1]
        self.piles[move[0] - 1] -= m

## Rozegranie gry

Funkcja pomocnicza odpowiadająca za uruchamienie daną ilość razy (`how_many_times`), danej gry (`game_type`), z danym algorytmem AI (`ai_algorithm`) i daną głębokością (`depth`). Co wykonanie zmienia gracza rozpoczynającego.

In [3]:
from easyAI import AI_Player, Negamax

def play_nim(how_many_times, game_type, ai_algorithm, depth):
    ai = ai_algorithm(depth)
    for i in range(how_many_times):
        game = game_type([AI_Player(ai), AI_Player(ai)], current_player=i%2+1)
        game.play()
        print("player %d wins" % game.current_player)

Najpierw uruchamiamy grę 10 razy w wersji deterministycznej oraz 10 razy w wersji probabilistycznej z algorytmem Negamax (głębokość ustawiona na 2). Następnie powtarzamy eksperyment z głębokością ustawioną na 5.

In [4]:
play_nim(10, Nim, Negamax, 2)
play_nim(10, MissNim, Negamax, 2)
play_nim(10, Nim, Negamax, 5)
play_nim(10, MissNim, Negamax, 5)

TypeError: Nim.__init__() got an unexpected keyword argument 'current_player'